In [2]:
import pymongo
from pymongo import MongoClient
import yaml
import pandas as pd
import pprint
import json

In [3]:
with open('C:/Users/cc843/Desktop/辛旻宗/주식프로젝트/key/config.yaml', encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']
MONGO_client = _cfg['mongodb']

In [4]:
# 몽고 db설정
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client["root"]  # db이름
users_collection = db["stock"]  # 폴더이름

query = {"output1.hts_kor_isnm":"삼성전자"}

In [10]:
# data = {}
# for document in users_collection.find(query):
#   pprint.pprint(document)

{'_id': ObjectId('660e1bfa3745ce33513a56e9'),
 'msg1': '정상처리 되었습니다.',
 'msg_cd': 'MCA00000',
 'output1': {'acml_tr_pbmn': '1408578505000',
             'acml_vol': '16569442',
             'askp': '85000',
             'bidp': '84900',
             'cpfn': '7780',
             'eps': '2131.00',
             'hts_avls': '5074315',
             'hts_kor_isnm': '삼성전자',
             'itewhol_loan_rmnd_ratem name': '0.11',
             'lstn_stcn': '5969782550',
             'pbr': '1.63',
             'per': '39.89',
             'prdy_ctrt': '1.07',
             'prdy_vol': '30493348',
             'prdy_vrss': '900',
             'prdy_vrss_sign': '2',
             'prdy_vrss_vol': '-13923906',
             'stck_fcam': '100',
             'stck_hgpr': '85500',
             'stck_llam': '58900',
             'stck_lwpr': '84300',
             'stck_mxpr': '109300',
             'stck_oprc': '85200',
             'stck_prdy_clpr': '84100',
             'stck_prdy_hgpr': '85000',
         

In [30]:
# MongoDB에서 데이터 가져오기
query = {"output2.stck_bsop_date": {"$exists": True}}  # stck_bsop_date 필드가 있는 경우
#projection = {"output2.stck_bsop_date": 1}  # 필요한 필드만 선택 (원하는 다른 필드도 선택 가능)
cursor = users_collection.find(query) #projection)

# 결과를 리스트로 변환
data_list = list(cursor)

df_list = []

for data in data_list:
    # output1과 output2를 각각 DataFrame으로 변환하여 추가
    output1_df = pd.DataFrame(data['output1'], index=[0])
    output2_df = pd.DataFrame(data['output2'])
    # 기존 데이터와 output1, output2 데이터를 합쳐서 하나의 DataFrame으로 만듭니다.
    result_df = pd.concat([pd.DataFrame(data, index=[0]), output1_df, output2_df], axis=1)
    # output1과 output2를 제거합니다.
    result_df = result_df.drop(columns=['output1', 'output2','rt_cd','msg_cd','msg1'])
    #df_list.append(result_df)

# 결과 확인
result_df


,_id,prdy_vrss,prdy_vrss_sign,prdy_ctrt,stck_prdy_clpr,acml_vol,acml_tr_pbmn,hts_kor_isnm,stck_prpr,stck_shrn_iscd,...,stck_hgpr,stck_lwpr,acml_vol,acml_tr_pbmn,flng_cls_code,prtt_rate,mod_yn,prdy_vrss_sign,prdy_vrss,revl_issu_reas
0,660e1c293745ce33513a57a5,900,2,1.07,84100,16581064,1409565766800,삼성전자,85000,005930,...,85500,84300,16581064,1409565766800,00,0.00,N,2,900,


In [36]:
print(result_df)

                        _id prdy_vrss prdy_vrss_sign prdy_ctrt stck_prdy_clpr  \
0  660e1c293745ce33513a57a5       900              2      1.07          84100   

   acml_vol   acml_tr_pbmn hts_kor_isnm stck_prpr stck_shrn_iscd  ...  \
0  16581064  1409565766800         삼성전자     85000         005930  ...   

  stck_hgpr stck_lwpr  acml_vol   acml_tr_pbmn flng_cls_code prtt_rate mod_yn  \
0     85500     84300  16581064  1409565766800            00      0.00      N   

  prdy_vrss_sign prdy_vrss revl_issu_reas  
0              2       900                 

[1 rows x 44 columns]
